#Netflix_titles Data Clean

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
print('Completed')

Completed


In [3]:
# Read in the data
Netflix_titles= pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTiy6szk-aG7h9phqKJe8TfxYqITUYAi1LNXRZHg3YYEfWdUhxkGmb-ZUDrgYfupAQnylK6t3aCS_iR/pub?gid=171732743&single=true&output=csv')
Netflix_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [4]:
#Make a copy of data
df= Netflix_titles.copy()

In [5]:
# Check the info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [6]:
# Change show_id form
df['show_id']=df['show_id'].str.replace('s','')

In [7]:
# Check uni values
df.nunique()

show_id         8807
type               2
title           8807
director        4528
cast            7692
country          748
date_added      1714
release_year      74
rating            17
duration         220
listed_in        514
description     8775
dtype: int64

In [9]:
# Check any null values
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

 With director & cast We won't be using them, so discard them.

 Country: fill it later 

 Date_added: if you look closely, you'll notice that it's safe to do a ffill. 
 
 Plus, the rows with missing values for this column are the last in the dataset

 Just 4 rating missing so We will search and fill it.

In [10]:
# Drop the columns we don't need
df.drop(['director','cast'], axis=1, inplace=True)

In [12]:
# We fill the missing values for country with the mode(df.mode Return the highest frequency value)
df['country'].fillna(df['country'].mode()[0], inplace=True)


In [13]:
# Now We forward fill the holes in date_added, which are the last records in the dataset
# ‘ffill’ stands for ‘forward fill’ and will propagate last valid observation forward
df['date_added'] = df['date_added'].ffill()

In [14]:
# Check any null values
df.isnull().sum()

show_id         0
type            0
title           0
country         0
date_added      0
release_year    0
rating          4
duration        3
listed_in       0
description     0
dtype: int64

In [16]:
# Check the rating column with null rating
df[df['rating'].isnull()]

,show_id,type,title,country,date_added,release_year,rating,duration,listed_in,description
5989,5990,Movie,13TH: A Conversation with Oprah Winfrey & Ava ...,United States,"January 26, 2017",2017,NaN,37 min,Movies,Oprah Winfrey sits down with director Ava DuVe...
6827,6828,TV Show,Gargantia on the Verdurous Planet,Japan,"December 1, 2016",2013,NaN,1 Season,"Anime Series, International TV Shows","After falling through a wormhole, a space-dwel..."
7312,7313,TV Show,Little Lunch,Australia,"February 1, 2018",2015,NaN,1 Season,"Kids' TV, TV Comedies","Adopting a child's perspective, this show take..."
7537,7538,Movie,My Honor Was Loyalty,Italy,"March 1, 2017",2015,NaN,115 min,Dramas,"Amid the chaos and horror of World War II, a c..."


In [17]:
# Fill the null with correct values from gg
replacements = {5990:'PG-13', 6828: 'TV-PG', 7313: 'TV-Y', 7538: 'PG-13'}
for show_id, rating in replacements.items():
  index=show_id-1
  df.iloc[index,6] = rating

In [18]:
df[df['show_id']=='6828']

,show_id,type,title,country,date_added,release_year,rating,duration,listed_in,description
6827,6828,TV Show,Gargantia on the Verdurous Planet,Japan,"December 1, 2016",2013,TV-PG,1 Season,"Anime Series, International TV Shows","After falling through a wormhole, a space-dwel..."


In [19]:
df.tail()

,show_id,type,title,country,date_added,release_year,rating,duration,listed_in,description
8802,8803,Movie,Zodiac,United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,8804,TV Show,Zombie Dumb,United States,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,8805,Movie,Zombieland,United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,8806,Movie,Zoom,United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."
8806,8807,Movie,Zubaan,India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...


In [20]:
# Check any null values
df.isnull().sum()

show_id         0
type            0
title           0
country         0
date_added      0
release_year    0
rating          0
duration        3
listed_in       0
description     0
dtype: int64

In [21]:
# Check duration values is null
df[df['duration'].isnull()]

,show_id,type,title,country,date_added,release_year,rating,duration,listed_in,description
5541,5542,Movie,Louis C.K. 2017,United States,"April 4, 2017",2017,74 min,NaN,Movies,"Louis C.K. muses on religion, eternal love, gi..."
5794,5795,Movie,Louis C.K.: Hilarious,United States,"September 16, 2016",2010,84 min,NaN,Movies,Emmy-winning comedy writer Louis C.K. brings h...
5813,5814,Movie,Louis C.K.: Live at the Comedy Store,United States,"August 15, 2016",2015,66 min,NaN,Movies,The comic puts his trademark hilarious/thought...


In [22]:
# Replace correct duration
correctduration = {5542:'74 min', 5795: '84 min', 5814: '66 min'}
for show_id, duration in correctduration.items():
  index=show_id-1
  df.iloc[index,7] = duration

In [23]:
# How many kind of rating on netflix 
print(df['rating'].unique())
print(len(df['rating'].unique()))

['PG-13' 'TV-MA' 'PG' 'TV-14' 'TV-PG' 'TV-Y' 'TV-Y7' 'R' 'TV-G' 'G'
 'NC-17' '74 min' '84 min' '66 min' 'NR' 'TV-Y7-FV' 'UR']
17


In [24]:
#Check unusual data
df[df['rating']=='UR']

,show_id,type,title,country,date_added,release_year,rating,duration,listed_in,description
7058,7059,Movie,Immoral Tales,France,"June 6, 2019",1974,UR,103 min,"Dramas, International Movies, Romantic Movies",This anthology illustrates the timeless nature...
7988,7989,Movie,Sex Doll,"United Kingdom, France","July 10, 2017",2016,UR,103 min,"Dramas, International Movies, Romantic Movies",A high-end call girl falls for a handsome man ...
8790,8791,Movie,You Don't Mess with the Zohan,United States,"September 1, 2019",2008,UR,113 min,"Action & Adventure, Comedies",An Israeli counterterrorism soldier with a sec...


In [25]:
# Replace correct rating 
correctrating = {5542:'TV-MA', 5795: 'TV-MA', 5814: 'TV-MA', 7059: 'TV-MA', 7989: 'TV-MA', 8791: 'PG-13'}
for show_id, rating in correctrating.items():
  index=show_id-1
  df.iloc[index,6] = rating

In [26]:
# Check data type 
df.dtypes

show_id         object
type            object
title           object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object

In [27]:
# Convert the values of date_added column to dates
df['date_added'] = pd.to_datetime(df['date_added'])

In [29]:
# Convert show_id column to int
df['show_id'] = pd.to_numeric(df['show_id'])

In [31]:
# Check any null values
df.isnull().sum()

show_id         0
type            0
title           0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

In [33]:
df['type'].unique()

array(['Movie', 'TV Show'], dtype=object)

In [34]:
# Check any null values
df.isnull().sum()

show_id         0
type            0
title           0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

In [35]:
# Replace correct rating 
correctrating = {5542:'TV-MA', 5795: 'TV-MA', 5814: 'TV-MA', 7059: 'TV-MA', 7989: 'TV-MA', 8791: 'PG-13'}
for show_id, rating in correctrating.items():
  index=show_id-1
  df.iloc[index,6] = rating

In [36]:
# How many kind of rating on netflix 
print(df['rating'].unique())
print(len(df['rating'].unique()))

['PG-13' 'TV-MA' 'PG' 'TV-14' 'TV-PG' 'TV-Y' 'TV-Y7' 'R' 'TV-G' 'G'
 'NC-17' 'NR' 'TV-Y7-FV']
13


In [38]:
# Replace rating by Age
df['rating']=df['rating'].replace('TV-Y7-FV','Older Kids')
df['rating']=df['rating'].replace('TV-Y7','Older Kids')
df['rating']=df['rating'].replace('TV-Y','Little Kids')
df['rating']=df['rating'].replace('TV-PG','Older Kids')
df['rating']=df['rating'].replace('TV-G','Little Kids')
df['rating']=df['rating'].replace('TV-14','Teens')
df['rating']=df['rating'].replace('TV-MA','Mature')
df['rating']=df['rating'].replace('NC-17','Mature')
df['rating']=df['rating'].replace('PG-13','Teens')
df['rating']=df['rating'].replace('PG','Older Kids')
df['rating']=df['rating'].replace('NR','Not Rated')
df['rating']=df['rating'].replace('G','Little Kids')
df['rating']=df['rating'].replace('R','Mature')

In [39]:
# Save as a excel file
df.to_excel('project_1.xlsx', index=False)

In [41]:
df_movie = df[df.type == 'Movie'].copy()

In [42]:
# Save as a csv file
df_movie.to_csv('movie.csv', index=False)

In [43]:
df_TV = df[df.type == 'TV Show'].copy()

In [45]:
# Save as a csv file
df_TV.to_csv('TV.csv', index=False)

# Stack Country & Listed_in

Too much infomation in column listed_in, so we need to split the required data

In [57]:
df = Netflix_titles.copy()
# Change show_id form again
df['show_id']=df['show_id'].str.replace('s','')

In [58]:
df.set_index('show_id')

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
show_id,,,,,,,,,,,
1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...
8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...


In [59]:
# Drop unnecessary column
df.drop(['date_added','director','cast','type','release_year','listed_in','description','rating','duration'], axis=1, inplace=True)

In [60]:
#Split the required data
df_country = df.drop(['country'], axis=1).join(df['country'].str.split(',',expand=True).stack().reset_index(level=1, drop=True).rename('country'))

In [61]:
df_country.set_index('show_id')

,title,country
show_id,,
1,Dick Johnson Is Dead,United States
2,Blood & Water,South Africa
3,Ganglands,NaN
4,Jailbirds New Orleans,NaN
5,Kota Factory,India
...,...,...
8803,Zodiac,United States
8804,Zombie Dumb,NaN
8805,Zombieland,United States


In [62]:
# Creative Filter_country Table
df_country.to_csv('Filter_country.csv')

In [63]:
df = Netflix_titles.copy()
# Change show_id form again
df['show_id']=df['show_id'].str.replace('s','')

In [64]:
df.set_index('show_id')

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
show_id,,,,,,,,,,,
1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...
8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...


In [65]:
# Drop unnecessary column
df.drop(['date_added','director','cast','type','release_year','country','description','rating','duration'], axis=1, inplace=True)

In [66]:
# Split the required data
df_listed_in = df.drop(['listed_in'], axis=1).join(df['listed_in'].str.split(',',expand=True).stack().reset_index(level=1, drop=True).rename('listed_in'))

In [67]:
df_listed_in.set_index('show_id')

,title,listed_in
show_id,,
1,Dick Johnson Is Dead,Documentaries
2,Blood & Water,International TV Shows
2,Blood & Water,TV Dramas
2,Blood & Water,TV Mysteries
3,Ganglands,Crime TV Shows
...,...,...
8806,Zoom,Children & Family Movies
8806,Zoom,Comedies
8807,Zubaan,Dramas


In [68]:
# Creative Filter_listed_in Table
df_listed_in.to_csv('Filter_listed_in.csv')

Done!